In [2]:
import threading
import time
import multiprocessing

We'll test whether:
1) there is parallilisation
2) if there is an overhead in copying
3) whether overwiting happens
4) how data can be transferred to global memory
5) What the option are (daemon, ...), joining, ...

For some basic info on threading vs multiprocessing, and locking, see [this link](https://timber.io/blog/multiprocessing-vs-multithreading-in-python-what-you-need-to-know).

Python has a [Global Interpreter Lock (GIL)](https://realpython.com/python-gil/).

In [7]:
def test(i):
    print(f"Starting process {i}.")
    time.sleep(4.)
    print(f"Finishing process {i}.")

for j in range(10):
    t = threading.Thread(target = test, args = (j,))
    t.start()

Starting process 0.Starting process 1.

Starting process 2.
Starting process 3.
Starting process 4.
Starting process 5.
Starting process 6.
Starting process 7.
Starting process 8.Starting process 9.

Finishing process 0.Finishing process 1.

Finishing process 2.
Finishing process 3.
Finishing process 4.
Finishing process 5.
Finishing process 6.
Finishing process 7.
Finishing process 8.
Finishing process 9.


They seem to be run at the same time... In the explanations it seemed that this wouldn't be the case? Or is it perhaps because during the waiting, the processor can switch tasks?

In [11]:
def test2(i):
    print(f"Starting process {i}.")
    tm = time.time()
    k = 0
    for k in range(1000000):
        k += 1
    print(f"Finishing process {i}, took {time.time() - tm} seconds.")

overall_tm = time.time()
for j in range(10):
    t = threading.Thread(target = test2, args = (j,))
    t.start()
    t.join()
print(f"Total time: {time.time() - overall_tm}.")    

Starting process 0.
Finishing process 0, took 0.1183311939239502 seconds.
Starting process 1.
Finishing process 1, took 0.18120050430297852 seconds.
Starting process 2.
Finishing process 2, took 0.11119675636291504 seconds.
Starting process 3.
Finishing process 3, took 0.10800290107727051 seconds.
Starting process 4.
Finishing process 4, took 0.1157844066619873 seconds.
Starting process 5.
Finishing process 5, took 0.10674524307250977 seconds.
Starting process 6.
Finishing process 6, took 0.10868406295776367 seconds.
Starting process 7.
Finishing process 7, took 0.0968780517578125 seconds.
Starting process 8.
Finishing process 8, took 0.149216890335083 seconds.
Starting process 9.
Finishing process 9, took 0.13461828231811523 seconds.
Total time: 1.286689043045044.


This seems to be additive. Now let's try with [multiprocessing](https://www.geeksforgeeks.org/multiprocessing-python-set-1/).

In [12]:
overall_tm = time.time()
for j in range(10):
    p = multiprocessing.Process(target = test2, args = (j,))
    p.start()
p.join()
print(f"Total time: {time.time() - overall_tm}.")    

Total time: 1.7222249507904053.


There's no print outpur, and total time seems to be higher. What happened?